# $\texttt{feyntrop}$ tutorial: 2L_3pt diagram

import the $\texttt{feyntrop}$ module, assuming $\texttt{feyntrop.so}$ and $\texttt{pytrop.py}$ are in the working directory 

In [1]:
from pytrop import *

## The graph

* We specify the graph from the tutorial of the paper via egdes $e$ and edge weights $\nu_e$.
* An edge $e = (v_i,v_j)$ is given by an unordered pair of vertices $v_i$ and $v_j$.
* The edge weight $\nu_e$ is written after the edge, so an entry in $\texttt{graph}$ has the form $((v_i,v_j), \nu_e)$.
* Note the following convention: external vertices must come first, so $V_\text{ext} = \{0,1,2\}$ label external partcles, and $V_\text{int} = \{3\}$.

In [2]:
graph = [((0,1), 1), ((1,3), 1), ((3,2), 1), ((2,0), 1), ((0,3), 1)]

## Kinematics

* Every edge in $\texttt{graph}$ has an associated squared mass.
* For simplicity we set $m_e^2 = 0.2$ for all $e$.

In [3]:
masses = [0.2] * len(graph)

* Next we provide values for the momentum configuration.
* The values for squared momenta $p_0^2, \ldots, p_{|V_\text{ext}|-2}^2$ are denoted by $(\texttt{sp}[v,v], \text{value})$, where $\texttt{sp}$ stands for scalar product. In this case $|V_\text{ext}| = 3$.
* The values for Mandelstam variables $s_{ij}=(p_i+p_j)^2$ with $0 \leq i < j \leq |V_\text{ext}|-2$ are written as $(\texttt{s}[i,j], \text{value})$.

In [4]:
momentum_vars = [(sp[0,0], 0), (sp[1,1], 0), (s[0,1], 1)]

## Additional settings

* $\texttt{D0}$ is the integer part of the spacetime dimension $D = D_0 - 2\epsilon$.
* We expand up to but not including $\texttt{eps}$_$\texttt{order}$.
* $\texttt{Lambda}$ denotes the deformation tuning parameter $\lambda$.
* $\texttt{N}$ is the number of Monte Carlo sampling points.

In [5]:
D0 = 2
eps_order = 5
Lambda = 8.9
N = int(1e8)

## Tropical integration

* $\texttt{trop}$_$\texttt{res}$ is the value of the Feynman integral, without any prefactor.
* $\texttt{Itr}$ is the normalization factor in the tropical measure.

In [6]:
trop_res, Itr = tropical_integration(graph, masses, momentum_vars, D0, eps_order, Lambda, N)

Prefactor: gamma(2*eps + 3).
Analytic continuation: activated.
Generalized permutahedron property: fulfilled.
Started integrating using 8 threads and N = 1e+08 points.
Finished in 60.2817 seconds = 0.0167449 hours.

-- eps^0: [-46.600 +/- 0.042] + [87.084 +/- 0.039] * i
-- eps^1: [-273.66 +/- 0.18] + [105.86 +/- 0.18] * i
-- eps^2: [-420.01 +/- 0.41] + [-182.71 +/- 0.42] * i
-- eps^3: [-167.58 +/- 0.68] + [-474.64 +/- 0.68] * i
-- eps^4: [201.97 +/- 0.85] + [-388.91 +/- 0.84] * i


* This yields the $\epsilon$-expansion with prefactor $\frac{\Gamma(\omega)}{\Gamma(\nu_1) \cdots \Gamma(\nu_E)}$ included, where $\omega$ is the superficial degree of divergence and $E$ is the number of edges.

In [7]:
eps_expansion(trop_res, graph, D0)

174.1685009*I - 93.20036813 + eps*(-719.3237731 + 533.1548237*I) + eps**2*(-2082.478069 + 459.5110204*I) + eps**3*(-3464.274112 - 695.2953202*I) + eps**4*(-3739.117761 - 2633.207558*I) + O(eps**5)